# Çol Değişkneli Aykırı Gözlem Analizi

### Local Outlier Factor

Gözlemleri bulundukaları konumda yoğunluk tabalı skorlayarak buna aykırı değer olabilecek değerleri tanımlayabilmemize imkan sağlıyor.

Bir noktanın lokal yoğunlupu bu noktanın komşuları ile karşılaştırılıyor. 
Eğer bir nokta komşularının yoğunlıuğundan anlamlı şekilde düşük ise bu nokta komşularından 
daha seyrek bir bölgede bulunuyordur yorumunu yapılabiliyor.
Dolayısıyka burada bir komşuluk yapısı söz konusu bir değerin çevresi yoğun değilse demek ki bu değer aykırı değerdir şeklinde değerlendiriliyor

In [1]:
import seaborn as sns
import pandas as pd
dt=sns.load_dataset("diamonds")
df=dt.copy()
df=df.select_dtypes(include=["int64","float64"])
df=df.dropna()
df.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   carat   53940 non-null  float64
 1   depth   53940 non-null  float64
 2   table   53940 non-null  float64
 3   price   53940 non-null  int64  
 4   x       53940 non-null  float64
 5   y       53940 non-null  float64
 6   z       53940 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 2.9 MB


### Silme

In [3]:
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

In [12]:
x=LocalOutlierFactor(n_neighbors=20, contamination=0.1)
x

LocalOutlierFactor(contamination=0.1)

In [19]:
x.fit_predict(df)[0:20]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,
        1,  1,  1])

In [6]:
d=x.negative_outlier_factor_
d # bu her bir değerin yoğunlıuk skoru

array([-1.58352526, -1.59732899, -1.62278873, ..., -1.06721815,
       -1.00809552, -1.00849038])

In [7]:
np.sort(d)[0:10]
#burada bu yoğunluk değerlerinin sıralayıp bakıyoruz

array([-8.60430658, -8.20889984, -5.86084355, -4.98415175, -4.81502092,
       -4.81502092, -4.61522833, -4.37081214, -4.29842288, -4.10492387])

In [8]:
#normald eddah dikkatli olarak sınır değeri belirlenir ancak biz burada 13. değeri eşik kabul edipi işlemlerimizi yapıcaz
esik=np.sort(d)[13]

In [26]:
non_outlier=d>esik
non_outlier

array([ True,  True,  True, ...,  True,  True,  True])

In [27]:
df_new=df[non_outlier]
df_new
#bunlar esik değer içerisinde bulunanlar yani burarda aykırı gözlemlerden kurtukmuş olduk :))

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74


### Baskılama

In [28]:
baski_değeri = df[d==esik] # bu eşik değer gözlemi
baski_değeri

,carat,depth,table,price,x,y,z
31230,0.45,68.6,57.0,756,4.73,4.5,3.19


In [29]:
outliers=df[~non_outlier] #aykır olan gözlemleri elde ettik

In [30]:
# hocam atayacağımız tek bir değer değil. çoklu değişkenle çalışıyoruz
# eşik depğer gözlemindelüki değerleri ayj-kırı gözlemlerdeki yerkler atıyıcaz bu yüzden index sıkıntısı yaşamamak için
# önce aykırı değerleri ve bizim eşik gözlimi şndex siz arraya çevirmemmiz sonra atama yapmamız gerekiyor

In [35]:
indexlesness=outliers.to_records(index=False)
indexlesness

rec.array([(1.  , 44. , 53. ,  4032, 6.31,  6.24,  4.12),
           (1.09, 43. , 54. ,  4778, 6.53,  6.55,  4.12),
           (2.  , 58.9, 57. , 12210, 8.09, 58.9 ,  8.06),
           (0.45, 68.6, 57. ,   756, 4.73,  4.5 ,  3.19),
           (0.29, 62.8, 44. ,   474, 4.2 ,  4.24,  2.65),
           (0.3 , 51. , 67. ,   945, 4.67,  4.62,  2.37),
           (0.73, 70.8, 55. ,  1049, 5.51,  5.34,  3.84),
           (1.03, 78.2, 54. ,  1262, 5.72,  5.59,  4.42),
           (0.7 , 71.6, 55. ,  1696, 5.47,  5.28,  3.85),
           (0.51, 61.8, 54.7,  1970, 5.12,  5.15, 31.8 ),
           (0.51, 61.8, 55. ,  2075, 5.15, 31.8 ,  5.12),
           (0.81, 68.8, 79. ,  2301, 5.26,  5.2 ,  3.58),
           (0.5 , 79. , 73. ,  2579, 5.21,  5.18,  4.09),
           (0.5 , 79. , 73. ,  2579, 5.21,  5.18,  4.09)],
          dtype=[('carat', '<f8'), ('depth', '<f8'), ('table', '<f8'), ('price', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8')])

In [38]:
indexlesness[:]=baski_değeri.to_records(index=False)
indexlesness

rec.array([(0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19)],
          dtype=[('carat', '<f8'), ('depth', '<f8'), ('table', '<f8'), ('price', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8')])

In [39]:
#hocam iyi güzel değişktirdik atadık ama asıl data frame de bir değişiklik olmadı
# bunun için önce index siz arrayi bir data frame'e dönüştürücez daha sonra bunu asıl data frame'e atıyıcaz

In [41]:
df[~non_outlier]=pd.DataFrame(indexlesness, index=df[~non_outlier].index)

In [ ]:
#ancak indexsiz arkadaşı data frame çeviriyoruz index olarak bizim data frema mizmizin indexlerini atıyorz
#yanşi buü arrayi bir data frame yap indexlerini bu date farmein indexleri gibi yap :))